In [1]:
import numpy as np
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier

import mean_variance

In [45]:
# Reads risk/return inputs and correlation matrix

return_risk_df = pd.read_excel('..\docs\optimization_assumptions_2022.xlsx',
                                sheet_name='return_risk', index_col=0)

corr_df = pd.read_excel('..\docs\optimization_assumptions_2022.xlsx',
                        sheet_name='corr', index_col=0)

expected_returns = return_risk_df.exp_return

In [5]:
# Produces covariance matrix from correlation matrix and standard deviation

cov_df = corr_df.copy()

for i in np.arange(return_risk_df.exp_risk.shape[0]):

    for j in np.arange(return_risk_df.exp_risk.shape[0]):

        cov_df.iloc[i,j] = cov_df.iloc[i,j] * return_risk_df.risk[i] * return_risk_df.risk[j]


In [18]:
ef = EfficientFrontier(expected_returns, cov_df, weight_bounds=(0,0.3))  # setup
clean_weights = ef.efficient_risk(0.12)  # find the portfolio that maximizes volatility
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

OrderedDict([('treasuries', 0.0), ('credit', 0.10406), ('cdn_equity', 0.0), ('gbl_equity', 0.14797), ('em_equity', 0.0), ('alt_assets', 0.14797), ('alt_strats', 0.3), ('pvt_equity', 0.3)])


In [29]:
ef.portfolio_performance(verbose=True)

Expected annual return: 7.6%
Annual volatility: 12.0%
Sharpe Ratio: 0.47


(0.0758780754870514, 0.11999999914095776, 0.465650632392208)

In [47]:
sims = 1000     # Number of simulations to run
max_w = 0.3     # Maximum weight of an asset class 
ret_sd = 0.01   # Standard deviation that long term expected annual returns varies by
risk_sd = 0.01  # Standard deviation that long term expected annual volatility varies by

total_weights = []
total_performance = []

for sim in np.arange(sims):

    #Adds a layer of randomness to returns
    expected_returns = [er + ret_sd * np.random.randn() for er in return_risk_df.exp_return]

    #Adds a layer of randomness to risk
    expected_risk = [er + risk_sd * np.random.randn() for er in return_risk_df.exp_risk]

     # Produces covariance matrix from correlation matrix and standard deviation
    cov_df = corr_df.copy()
    
    for i in np.arange(return_risk_df.exp_risk.shape[0]):

        for j in np.arange(return_risk_df.exp_risk.shape[0]):

            cov_df.iloc[i,j] = cov_df.iloc[i,j] * return_risk_df.exp_risk[i] * return_risk_df.exp_risk[j]
    
    # Setup mean variance
    ef = EfficientFrontier(expected_returns, cov_df, weight_bounds=(0,max_w)) 
    clean_weights = ef.efficient_risk(0.12)  # find the portfolio that minimises volatility and L2_reg
    cleaned_weights = ef.clean_weights()

    # Saves each simulation's portfolios weights and performance 
    total_weights.append(list(cleaned_weights.values()))
    total_performance.append(list(ef.portfolio_performance(verbose=False)))



In [63]:
# Takes the average of the simulation weights for each asset class 
average_weights = np.mean(total_weights, axis=0)

# Prints the weights
for i in np.arange(average_weights.shape[0]):
    print(f'{corr_df.columns[i]}: {np.round(average_weights[i]*100, 2)}')

treasuries: 5.38
credit: 9.8
cdn_equity: 4.99
gbl_equity: 13.42
em_equity: 3.02
alt_assets: 13.76
alt_strats: 22.82
pvt_equity: 26.8


In [71]:
# Takes the average of the simulation performance for each metric
average_performance = np.mean(total_performance, axis=0)

# Prints the metrics
print(f'Expected return: {np.round(average_performance[0]*100,2)}')
print(f'Expected risk: {np.round(average_performance[1]*100,2)}')
print(f'Expected Sharpe: {np.round(average_performance[2],3)}')

Expected return: 7.88
Expected risk: 12.0
Expected Sharpe: 0.49


In [2]:
mean_variance.algorithm()

WEIGHTS
treasuries: 5.09
credit: 9.52
cdn_equity: 4.67
gbl_equity: 14.39
em_equity: 2.68
alt_assets: 14.11
alt_strats: 22.9
pvt_equity: 26.64

PERFORMANCE
Expected return: 7.89
Expected risk: 12.0
Expected Sharpe: 0.491
